In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
from tensorflow import keras
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.regularizers import l2
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
import warnings 
warnings.filterwarnings('ignore')
import os
from tensorflow.keras.models import load_model
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, RegressorMixin
import chess as chess

In [3]:
#Reading in our training data
df = pd.read_csv("/Users/viktorsjoberg/Desktop/Chess/data/chessData.csv")
#Reading in our validation data
Edf = pd.read_csv("/Users/viktorsjoberg/Desktop/Chess/data/random_evals.csv")

In [15]:
df.head(5)

,FEN,Evaluation
0,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,-10
1,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...,+56
2,rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...,-9
3,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...,+52
4,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPPN1PPP/R1BQK...,-26


In [16]:
Edf.head(5)

,FEN,Evaluation
0,rnbqkb1r/pppppppp/B4n2/8/4P3/8/PPPP1PPP/RNBQK1...,-459
1,rnbqkb1r/pppppppp/5n2/1B6/4P3/8/PPPP1PPP/RNBQK...,-125
2,rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBN...,+198
3,rnbqkb1r/pppppppp/5n2/8/4P3/7N/PPPP1PPP/RNBQKB...,-155
4,rnbqkbnr/ppppp1pp/8/5p2/4P3/8/PPPP1PPP/RNBQKBN...,+209


In [5]:
#Initiating our FEN to input Vector Function, which will output a (64,1) vector of our chess position.

def FENtoVEC (FEN):
    pieces = {"r":5,"n":3,"b":3.5,"q":9.5,"k":20,"p":1,"R":-5,"N":-3,"B":-3.5,"Q":-9.5,"K":-20,"P":-1}
    FEN = list(str(FEN.split()[0]))
    VEC = []
    for i in range(len(FEN)):
        if FEN[i] == "/":
            continue
        if FEN[i] in pieces:
            VEC.append(pieces[FEN[i]])
        else:
            em = [VEC.append(0) for i in range(int(FEN[i]))]

    return VEC





In [6]:
evl_strings = df["Evaluation"].tolist()
Eboard = Edf["FEN"].tolist()
# Function to clean and convert evaluation strings to float
def clean_and_convert(evl_str):
    # Remove non-numeric characters except for '-' (negative sign)
    cleaned_str = ''.join(filter(lambda x: x.isdigit() or x == '-', evl_str))

    # Handle Byte Order Mark (BOM) and other non-standard characters
    cleaned_str = cleaned_str.replace('\ufeff', '').replace('+', '')

    # Convert to float
    try:
        return float(cleaned_str)
    except ValueError:
        # Handle cases where conversion fails
        return None

# Apply the function to all evaluation strings
evl = np.asarray([clean_and_convert(i) for i in evl_strings])

# Filter out None values if any
evl = np.array([e for e in evl if e is not None])
NNtrainX = np.asarray([FENtoVEC(state) for state in Eboard])
NNtrainy = evl

In [7]:

Eevl = np.asarray([float(i.strip("#+")) for i in Edf["Evaluation"].tolist()])
for i in range(len(Eevl)):
    if Eevl[i] > 1000:
        Eevl[i] = float(1000)
    if Eevl.item(i) < -1000:
        Eevl[i] = float(1000)
    Eevl[i] = Eevl[i]/1000
NNvalX = np.asarray([FENtoVEC(state) for state in Eboard])
NNvaly = Eevl

In [104]:
l2_reg = 0.001

model = Sequential()
model.add(Dense(64, input_dim=64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='tanh'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

In [106]:

# Parameter grid
param_grid = [0.001, 0.01, 1]  # Adjust as needed

# GridSearchCV
grid = GridSearchCV(model, param_grid=param_grid, n_jobs=-1, cv=5, scoring='neg_mean_squared_error')
grid_result = grid.fit(NNtrainX, NNtrainy)

# Print results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

TypeError: Cannot clone object '<keras.src.engine.sequential.Sequential object at 0x424cc6ad0>' (type <class 'keras.src.engine.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [41]:
model.fit(NNtrainX, NNtrainy, epochs=500, batch_size=1000, verbose=0.1)

Epoch 1/500
Epoch 2/500
Epoch 3/500
Epoch 4/500
Epoch 5/500
Epoch 6/500
Epoch 7/500
Epoch 8/500
Epoch 9/500
Epoch 10/500
Epoch 11/500
Epoch 12/500
Epoch 13/500
Epoch 14/500
Epoch 15/500
Epoch 16/500
Epoch 17/500
Epoch 18/500
Epoch 19/500
Epoch 20/500
Epoch 21/500
Epoch 22/500
Epoch 23/500
Epoch 24/500
Epoch 25/500
Epoch 26/500
Epoch 27/500
Epoch 28/500
Epoch 29/500
Epoch 30/500
Epoch 31/500
Epoch 32/500
Epoch 33/500
Epoch 34/500
Epoch 35/500
Epoch 36/500
Epoch 37/500
Epoch 38/500
Epoch 39/500
Epoch 40/500
Epoch 41/500
Epoch 42/500
Epoch 43/500
Epoch 44/500
Epoch 45/500
Epoch 46/500
Epoch 47/500
Epoch 48/500
Epoch 49/500
Epoch 50/500
Epoch 51/500
Epoch 52/500
Epoch 53/500
Epoch 54/500
Epoch 55/500
Epoch 56/500
Epoch 57/500
Epoch 58/500
Epoch 59/500
Epoch 60/500
Epoch 61/500
Epoch 62/500
Epoch 63/500
Epoch 64/500
Epoch 65/500
Epoch 66/500
Epoch 67/500
Epoch 68/500
Epoch 69/500
Epoch 70/500
Epoch 71/500
Epoch 72/500
Epoch 73/500
Epoch 74/500
Epoch 75/500
Epoch 76/500
Epoch 77/500
Epoch 78

In [44]:
model.save('fit/model1.h5') 


In [9]:
model = load_model('/Users/viktorsjoberg/Desktop/Chess/fit/model1.h5')


2023-12-22 11:47:01.018162: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-12-22 11:47:01.018191: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-12-22 11:47:01.018200: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-12-22 11:47:01.018235: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-22 11:47:01.018259: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [53]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                4160      
                                                                 
 dense_13 (Dense)            (None, 64)                4160      
                                                                 
 dense_14 (Dense)            (None, 1)                 65        
                                                                 
Total params: 8385 (32.75 KB)
Trainable params: 8385 (32.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [64]:
RFtrain_X = pd.DataFrame(NNtrainX[:10000])
RFtrain_y = RFevl[:10000]
forest_model = RandomForestRegressor(n_estimators=200, max_depth= 32,random_state=1)
forest_model.fit(RFtrain_X, RFtrain_y)

RandomForestRegressor(max_depth=32, n_estimators=200, random_state=1)

In [78]:
results = model1.evaluate(NNvalX,NNvaly, batch_size=1)
print("Average Pawn loss over New dataset on the Neural Network",(results[0]**0.5)*10)

1000273/1000273 [==============================] - 1858s 2ms/step - loss: 1236640.0000 - mean_squared_error: 1.0086
Average Pawn loss over New dataset on the Neural Network 11120.431646298628


In [10]:

def evaluate_board():
    if board.is_checkmate():
        if board.turn:
            return -9999
        else:
            return 9999
    else:
        FENp = board.epd()
        inVEC = np.asarray([FENtoVEC(FENp)])
        evalu = model.predict(inVEC)[0]*10
        return evalu
board = chess.Board()
print(evaluate_board())
def alphabeta( alpha, beta, depthleft ):
    bestscore = -9999
    if( depthleft == 0 ):
        return quiesce( alpha, beta )
    for move in board.legal_moves:
        board.push(move)   
        score = -alphabeta( -beta, -alpha, depthleft - 1 )
        board.pop()
        if( score >= beta ):
            return score
        if( score > bestscore ):
            bestscore = score
        if( score > alpha ):
            alpha = score   
    return bestscore
def quiesce( alpha, beta ):
    stand_pat = evaluate_board()
    if( stand_pat >= beta ):
        return beta
    if( alpha < stand_pat ):
        alpha = stand_pat

    for move in board.legal_moves:
        if board.is_capture(move):
            board.push(move)        
            score = -quiesce( -beta, -alpha )
            board.pop()

            if( score >= beta ):
                return beta
            if( score > alpha ):
                alpha = score  
    return alpha
def selectmove(depth):
    bestMove = chess.Move.null()
    bestValue = -99999
    alpha = -100000
    beta = 100000
    for move in board.legal_moves:
        board.push(move)
        boardValue = -alphabeta(-beta, -alpha, depth-1)
        if boardValue > bestValue:
            bestValue = boardValue;
            bestMove = move
        if( boardValue > alpha ):
            alpha = boardValue
        board.pop()
    return bestMove

def evaluate_boardRF():
    if board.is_checkmate():
        if board.turn:
            return -9999
        else:
            return 9999
    else:
        FENp = board.epd()
        inVEC = np.asarray([FENtoVEC(FENp)])
        evalu = forest_model.predict(inVEC)[0]
        return evalu
def alphabetaRF( alpha, beta, depthleft ):
    bestscore = -9999
    if( depthleft == 0 ):
        return quiesceRF( alpha, beta )
    for move in board.legal_moves:
        board.push(move)   
        score = -alphabetaRF( -beta, -alpha, depthleft - 1 )
        board.pop()
        if( score >= beta ):
            return score
        if( score > bestscore ):
            bestscore = score
        if( score > alpha ):
            alpha = score   
    return bestscore
def quiesceRF( alpha, beta ):
    stand_pat = evaluate_boardRF()
    if( stand_pat >= beta ):
        return beta
    if( alpha < stand_pat ):
        alpha = stand_pat

    for move in board.legal_moves:
        if board.is_capture(move):
            board.push(move)        
            score = -quiesceRF( -beta, -alpha )
            board.pop()

            if( score >= beta ):
                return beta
            if( score > alpha ):
                alpha = score  
    return alpha
def selectmoveRF(depth):
    bestMove = chess.Move.null()
    bestValue = -99999
    alpha = -100000
    beta = 100000
    for move in board.legal_moves:
        board.push(move)
        boardValue = -alphabetaRF(-beta, -alpha, depth-1)
        if boardValue > bestValue:
            bestValue = boardValue;
            bestMove = move
        if( boardValue > alpha ):
            alpha = boardValue
        board.pop()
    return bestMove



for i in range(100):
    print(board)
    move = selectmove(2)
    print(move)
    board.push(move)
    print(evaluate_board())
    print(board)
    move = selectmoveRF(2)
    print(move)
    board.push(move)
    print(evaluate_boardRF())    

: 